In [1]:
"""
Plot 2D slice graphs.

"""

import h5py
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from dedalus.extras import plot_tools
import os
from os import listdir


In [2]:
#parameters

Lx=4
Ly=4
Lz=1
x_scale=256
y_scale=256
z_scale=64
y_slice_afterscale=10

y_slice_beforescale=y_slice_afterscale/y_scale*Ly

In [3]:
folder_dir = "snapshots"

file_paths = [os.path.join(folder_dir, file) for file in listdir(folder_dir) if os.path.isfile(os.path.join(folder_dir, file)) and file.endswith('.h5')]
file_paths.sort()
print(file_paths)

['snapshots/snapshots_s1.h5', 'snapshots/snapshots_s2.h5', 'snapshots/snapshots_s3.h5', 'snapshots/snapshots_s4.h5']


In [4]:
def quad_mesh(x, y, cut_x_edges=False, cut_y_edges=False):
    """
    Construct quadrilateral mesh arrays from two grids.
    Intended for use with e.g. plt.pcolor.

    Parameters
    ----------
    x : 1d array
        Grid for last axis of the mesh.
    y : 1d array
        Grid for first axis of the mesh.
    cut_x_edges, cut_y_edges : bool, optional
        True to truncate edge quadrilaterals at x/y grid edges.
        False (default) to center edge quadrilaterals at x/y grid edges.

    """

    # Get 1d vertex vectors
    xvert = get_1d_vertices(x, cut_edges=cut_x_edges)
    yvert = get_1d_vertices(y, cut_edges=cut_y_edges)
    # Reshape as multidimensional vectors
    xvert = reshape_vector(xvert, dim=2, axis=1)
    yvert = reshape_vector(yvert, dim=2, axis=0)
    # Broadcast up to arrays
    xmesh = xvert * np.ones_like(yvert)
    ymesh = yvert * np.ones_like(xvert)

    return xmesh, ymesh


def get_1d_vertices(grid, cut_edges=False):
    """
    Get vertices dividing a 1d grid.

    Parameters
    ----------
    grid : 1d array
        Grid.
    cut_edges : bool, optional
        True to set edge vertices at grid edges.
        False (default) to center edge segments at grid edges.

    """

    if len(grid.shape) > 1:
        raise ValueError("grid must be 1d array.")
    diff = np.diff(grid)
    vert = np.zeros(grid.size+1)
    # Interior vertices: halfway between points
    vert[1:-1] = grid[0:-1] + diff/2
    # Edge vertices: tight or reflect
    if cut_edges:
        vert[0] = grid[0]
        vert[-1] = grid[-1]
    else:
        vert[0] = grid[0] - diff[0]/2
        vert[-1] = grid[-1] + diff[-1]/2

    return vert

def reshape_vector(data, dim=2, axis=-1):
    """Reshape 1-dim array as a multidimensional vector."""
    # Build multidimensional shape
    shape = [1] * dim
    shape[axis] = data.size
    return data.reshape(shape)

In [5]:
n=0
for file in file_paths:
    with h5py.File(file, mode='r') as file:
            b = file['tasks']['buoyancy']
            st = file['scales/sim_time']
            xgrid = b.dims[1][0][:]
            zgrid = b.dims[3][0][:]
            xorder = np.argsort(xgrid)
            zorder = np.argsort(zgrid)
            xmesh, zmesh = quad_mesh(xgrid[xorder], zgrid[zorder])
            # Plot data
            for t in range(50):
                b_phase=b[t,:,y_slice_afterscale,:]
                b_phasef=np.transpose(b_phase)
                plt.figure(figsize=(20,5), dpi=200)
                plt.pcolormesh(xmesh, zmesh, b_phasef, shading='Auto', cmap='RdBu_r')
                plt.tick_params(length=0, width=0)
                plt.colorbar(label='buoyancy')
                plt.xlabel('x')
                plt.ylabel('z')

                # Add time title
                title = "t="+str(st[t])
                plt.title(title)
                # Save figure
                savename = "Plot_"+"%04d"%n+".png"
                savepath = 'frames/'+savename
                n=n+1
                plt.savefig(str(savepath), dpi=200,bbox_inches='tight')
                matplotlib.pyplot.close()


In [6]:
import matplotlib.animation as Animation
from PIL import Image


folder_dir = 'frames'

# Get a list of PNG file paths from the directory
file_paths = [os.path.join(folder_dir, f) for f in listdir(folder_dir) if os.path.isfile(os.path.join(folder_dir, f)) and f.endswith('.png')]

# Sort the list of file paths
file_paths.sort()

# Read the images using PIL
imgs = [Image.open(f) for f in file_paths]

fig = plt.figure(figsize=(16,4),dpi=200)
fig.patch.set_visible(False)
plt.axis('off')

# Wrap each image in a list to create a list of sequences of artists
imgs = [[plt.imshow(img, animated=True)] for img in imgs]

ani = Animation.ArtistAnimation(fig, imgs, interval=250, blit=True, repeat_delay=1000)

# Save the animation to a file
ani.save('dynamic_images.mp4',dpi=200)


2023-06-21 19:14:42,227 matplotlib.animation 0/1 INFO :: Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
2023-06-21 19:14:42,230 matplotlib.animation 0/1 INFO :: MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 3200x800 -pix_fmt rgba -r 4.0 -i pipe: -vcodec h264 -pix_fmt yuv420p -y dynamic_images.mp4


In [11]:
n=0
for file in file_paths:
    with h5py.File(file, mode='r') as file:
            b = file['tasks']['buoyancy']
            st = file['scales/sim_time']
            print(st)
            # Plot data
            for t in range(50):
                b_phase=b[t,:,y_slice_afterscale,:]
                b_phasef=np.transpose(b_phase)
                plt.figure(figsize=(20,5), dpi=200)
                xtemp =b.dims[1][0][:]
                x = xtemp.reshape(256,1)
                x = np.transpose(x)
                ztemp =b.dims[3][0][:]
                z = ztemp.reshape(64,1)
                one1=np.ones((64,1))
                one2=np.ones((1,256))
                xf = one1 @ x
                print(xf.shape)
                zf = z @ one2
                plt.pcolormesh(zf,xf, b_phasef,cmap='Reds',shading='nearest')
                plt.colorbar(label='buoyancy')
                plt.xlabel('x')
                plt.ylabel('z')

                # Add time title
                title = "t="+str(st[t])
                plt.title(title)
                # Save figure
                savename = "Plot_"+"%04d"%n+".png"
                savepath = 'frames/'+savename
                n=n+1
                plt.savefig(str(savepath), dpi=200,bbox_inches='tight')
                matplotlib.pyplot.close()


<HDF5 dataset "sim_time": shape (50,), type "<f8">
(64, 256)
(64, 256)
(64, 256)
(64, 256)
(64, 256)
(64, 256)
(64, 256)
(64, 256)
(64, 256)
(64, 256)
(64, 256)
(64, 256)
(64, 256)
(64, 256)
(64, 256)
(64, 256)
(64, 256)
(64, 256)
(64, 256)
(64, 256)


KeyboardInterrupt: 

In [3]:
file="snapshots/snapshots_s1.h5"
with h5py.File(file, mode='r') as file:
    b = file['tasks']['buoyancy']
    print(x)


[1.50590652e-04 1.35477166e-03 3.76023270e-03 7.36117881e-03
 1.21489350e-02 1.81119671e-02 2.52359097e-02 3.35036006e-02
 4.28951221e-02 5.33878494e-02 6.49565044e-02 7.75732174e-02
 9.12075934e-02 1.05826786e-01 1.21395577e-01 1.37876459e-01
 1.55229728e-01 1.73413579e-01 1.92384205e-01 2.12095904e-01
 2.32501190e-01 2.53550904e-01 2.75194335e-01 2.97379343e-01
 3.20052482e-01 3.43159130e-01 3.66643621e-01 3.90449380e-01
 4.14519056e-01 4.38794662e-01 4.63217718e-01 4.87729386e-01
 5.12270614e-01 5.36782282e-01 5.61205338e-01 5.85480944e-01
 6.09550620e-01 6.33356379e-01 6.56840870e-01 6.79947518e-01
 7.02620657e-01 7.24805665e-01 7.46449096e-01 7.67498810e-01
 7.87904096e-01 8.07615795e-01 8.26586421e-01 8.44770272e-01
 8.62123541e-01 8.78604423e-01 8.94173214e-01 9.08792407e-01
 9.22426783e-01 9.35043496e-01 9.46612151e-01 9.57104878e-01
 9.66496399e-01 9.74764090e-01 9.81888033e-01 9.87851065e-01
 9.92638821e-01 9.96239767e-01 9.98645228e-01 9.99849409e-01]


In [15]:
plt.contourf(b[-1,:,:,5])
plt.colorbar()
plt.show()

ValueError: Invalid dataset identifier (invalid dataset identifier)

In [17]:
                xa = np.linspace(-2,2,num=9)
                za = np.linspace(0,1,num=5)
                
                # create an index for each tick position
                xi = np.linspace(0,256,num=9)
                zi = np.linspace(0,64,num=5)
                plt.xticks(xi,xa)
                plt.yticks(zi,za)


<Closed HDF5 dataset>
